* Pure python implementation of decision trees

* Decision Tree is a binary tree that recursively splits a dataset until it is left with pure leaf nodes.
* Its a greedy algorithm

$$
H(S) = - \sum_{i=1}^{n} p(x_i) \log_2 p(x_i)
$$

$$
IG(T, A) = H(T) - \sum_{v \in \text{Values}(A)} \frac{|T_v|}{|T|} H(T_v)
$$

In [ ]:
# 1. Select a feature
# 2. Select feature value randomly from range
# 3. Calculate entropy (purity of sample)
#    a. Proportion of +(ve) examples -> p =  count(feature) groupby target filter target == 1 / sum(feature)
#    b. Proportion of -(ve) examples -> n =  1 - p
#    c. entropy = -plog2(p) - nlog2(n)
#    d. entropy (general) = sigma(-Pnlog2(Pn))
# 4. Split feature according to entropy and do 3. for each split (binary tree)

In [2]:
from sklearn.datasets import load_iris
import pandas as pd

# Load the Iris dataset
iris = load_iris()

# Convert to a pandas DataFrame for easier analysis
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Display the first few rows
#print(iris_df.head())

### Decision Tree Classifier

In [ ]:
def decision(feature: np.ndarray, target: np.ndarray) -> float:
    
    node_entropy = calcuate_entropy(feature,target)
    random_split = np.random.choice(feature_vector)




    
feature = SAMPLE_DATA.x1.values
target = SAMPLE_DATA.y.values   

In [ ]:
def calculate_entropy(feature,target):

    p = len(feature[target == True])/ len(feature)
    entropy = -(p*math.log2(p) - (1-p)*math.log2(1-p)) 
    
    return entropy
    

In [ ]:
print(calculate_entropy())

In [ ]:
class Node():
    
    def __init__(parent = None, left_child = None, right_child = None):
        self.parent = None
        self.left_child = left_child
        self.right_child = right_child
        
class Tree()

    def __init__(depth=0, root = None):
        self.depth = 0
        self.root = None
        
    def grow_tree
    
        
        